In [1]:
import numpy as np
import time

In [2]:
import sys
import os
project_root = os.path.abspath("../..")
sys.path.append(project_root)

In [3]:
from efficient_graph_gp.random_walk_samplers import Graph, RandomWalk

In [18]:
max_walk_length = 4
num_walks = 1000
p_halt = 0.1

In [ ]:
# Example 8x8 undirected graph adjacency matrix
W_full = np.array([
    [0, 1, 0, 1, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 1, 0],
    [0, 1, 0, 1, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0]
], dtype=float)

W_main = W_full[:-1, :-1]


In [13]:
I_main = np.eye(W_main.shape[0])
temp = I_main
M_main = [temp]

for i in range(1, max_walk_length):
    temp = temp @ W_main
    M_main.append(temp)
M_main = np.array(M_main)
    

In [14]:
M_main.shape

(4, 7, 7)

In [15]:
M_main_reshaped = np.transpose(M_main, (1, 2, 0))
print(M_main_reshaped.shape)  # Should output (7, 7, 4)

(7, 7, 4)


In [21]:
# Create Graph instance
main_graph = Graph(adjacency_matrix=W_main)

# Create RandomWalk instance
main_random_walk = RandomWalk(main_graph, seed=42)

# Perform the random walks and get the feature matrices as a NumPy array
main_feature_matrices = main_random_walk.get_random_walk_matrices(num_walks, p_halt, max_walk_length)

# Output the feature matrices
print("Feature matrices shape:", main_feature_matrices.shape)
for start_node in range(1):
    print(f"\nFeature matrix for start node {start_node}:")
    print(main_feature_matrices[start_node])


Feature matrices shape: (7, 7, 4)

Feature matrix for start node 0:
[[1.         0.         1.98518519 0.        ]
 [0.         1.02222222 0.         5.30041152]
 [0.         0.         2.1037037  0.        ]
 [0.         1.00444444 0.         4.88888889]
 [0.         0.         0.86666667 0.        ]
 [0.         0.         0.         1.90946502]
 [0.         0.         1.1037037  0.        ]]


In [8]:
def compute_C_naive(M):
    """
    Computes the matrix C where C[i, j, k] is the number of k-step paths
    from node i to node j that pass through node X, using naive convolution.

    Parameters:
    M (ndarray): An (N x P) matrix where M[i, p] is the number of paths
                 from node i to node X in p steps (p from 0 to P - 1).

    Returns:
    C (ndarray): An (N x N x P) matrix where C[i, j, k] is the number
                 of k-step paths from node i to node j passing through X.
                 k ranges from 0 to P - 1.
    """
    N, P = M.shape
    C = np.zeros((N, N, P))
    
    for i in range(N):
        for j in range(N):
            # Perform convolution of M[i, :] and M[j, :]
            C_ij = np.convolve(M[i, :], M[j, :])
            # Extract the relevant part of the convolution result (first P steps)
            C[i, j, :] = C_ij[:P]
    return C

def compute_C_fft(M):
    """
    Computes the matrix C where C[i, j, k] is the number of k-step paths
    from node i to node j that pass through node X, using FFT-based convolution.

    Parameters:
    M (ndarray): An (N x P) matrix where M[i, p] is the number of paths
                 from node i to node X in p steps (p from 0 to P - 1).

    Returns:
    C (ndarray): An (N x N x P) matrix where C[i, j, k] is the number
                 of k-step paths from node i to node j passing through X.
                 k ranges from 0 to P - 1.
    """
    N, P = M.shape
    # Determine the length for zero-padding, next power of 2 for efficiency
    L = 2 ** int(np.ceil(np.log2(2 * P - 1)))
    
    # Zero-pad the sequences to length L
    M_padded = np.zeros((N, L))
    M_padded[:, :P] = M
    
    # Compute the FFT along the last axis (step dimension)
    M_fft = np.fft.fft(M_padded, axis=1)
    
    # Compute the outer product of M_fft to get all pairwise products
    # Resulting shape will be (N, N, L)
    # Broadcasting is used to vectorize the computation
    C_fft = M_fft[:, np.newaxis, :] * M_fft[np.newaxis, :, :]
    
    # Compute the inverse FFT to get the convolutions in time domain
    C_padded = np.fft.ifft(C_fft, axis=2).real
    
    # Extract the relevant part of the convolution result (first P steps)
    C = C_padded[:, :, :P]
    
    return C

In [9]:
# Compute the full p-step walk matrices (feature matrices) via lazy update

# Time the computation
start_time = time.time()

num_nodes = full_adjacency_matrix.shape[0]
# Perform multiple walks for the new node
walk_matrix_new_node = full_random_walk._perform_multiple_walks(
    start_node=num_nodes - 1, num_walks=num_walks, p_halt=p_halt, max_walk_length=max_walk_length
)
# Pad the main feature matrices
main_feature_matrices_padded = np.pad(main_feature_matrices, ((0, 1), (0, 1), (0, 0)), mode='constant')
# Add the new node's walk matrix to the main feature matrices
main_feature_matrices_padded[:, -1, :] = main_feature_matrices_padded[-1, :, :] = walk_matrix_new_node
# Compute additional path count due to the new node
additional_path_counts = compute_C_naive(walk_matrix_new_node)
# Don't need to update the walk matrix for the new node
additional_path_counts[:, -1, :] = 0 
additional_path_counts[-1, :, :] = 0 
main_feature_matrices_padded += additional_path_counts
lazy_update_full_feature_matrices = main_feature_matrices_padded

end_time = time.time()

lazy_time = end_time - start_time
print(f"Lazy update time: {lazy_time:.4f} seconds")


Lazy update time: 0.0045 seconds


In [10]:
print(f"Naiive update time: {naiive_update_time:.4f} seconds")
print(f"Lazy update time: {lazy_time:.4f} seconds")

Naiive update time: 0.0072 seconds
Lazy update time: 0.0045 seconds


In [11]:
# Calculate the relative frobenius norm of the difference between the two methods
diff = np.linalg.norm(lazy_update_full_feature_matrices - full_feature_matrices) / np.linalg.norm(full_feature_matrices)
print(f"Relative Frobenius norm of the difference: {diff:.4f}")

Relative Frobenius norm of the difference: 0.0410
